In [2]:
import numpy as np
import os
import cv2
import random
import matplotlib.pyplot as plt

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

In [9]:
# Import necessary libraries
import numpy as np
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

# Load the pre-trained model
try:
    model = load_model('predictions/model/my_model.keras')  # Change the path to your model file
except Exception as e:
    raise RuntimeError(f"Error loading the model: {str(e)}")

# Define categories (these should match the classes in your model's output)
categories = [
    'BA- cellulitis',
    'BA-impetigo',
    'FU-athlete-foot',
    'FU-nail-fungus',
    'FU-ringworm',
    'PA-cutaneous-larva-migrans',
    'VI-chickenpox',
    'VI-chickenpox'
]

# Function to preprocess an image for the model
def prepare_image(image_path):
    """
    Preprocess the image for the ResNet50 model.
    - Load the image with target size 224x224.
    - Convert it to an array and preprocess it.
    """
    try:
        # Check if the image file exists
        if not os.path.exists(image_path):
            raise FileNotFoundError(f"Image not found at path: {image_path}")

        # Load the image with the target size
        img = load_img(image_path, target_size=(224, 224))
        img_array = img_to_array(img)
        
        # Expand dimensions to match model input and preprocess
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        
        return img_array
    except Exception as e:
        raise ValueError(f"Error processing the image: {str(e)}")

# Function to make predictions with the model
def predict_image(image_path):
    """
    Predict the class of the image.
    - Preprocess the image.
    - Predict using the model.
    - Return predicted class and confidence.
    """
    try:
        # Preprocess the image
        img_array = prepare_image(image_path)
        
        # Get predictions
        predictions = model.predict(img_array)
        
        # Get confidence scores for all classes
        confidence_scores = predictions[0]
        sorted_indices = np.argsort(confidence_scores)[::-1]  # Sort by confidence, highest first

        # Display top 3 predictions and their confidence scores
        top_predictions = [(categories[i], confidence_scores[i] * 100) for i in sorted_indices[:3]]
        
        # Determine the predicted class and confidence
        predicted_class_index = np.argmax(confidence_scores)
        confidence = confidence_scores[predicted_class_index]
        predicted_class = categories[predicted_class_index]

        # Print top predictions
        print("Top 3 Predictions:")
        for idx, (class_name, conf) in enumerate(top_predictions):
            print(f"{idx+1}. {class_name}: {conf:.2f}%")
        
        # Validate the predicted class
        if predicted_class not in categories:
            raise ValueError(f"Invalid class detected: {predicted_class}. Please check the image or model.")

        return predicted_class, confidence
    except Exception as e:
        raise RuntimeError(f"Error during prediction: {str(e)}")

# Function to display the image and prediction result with a 90% confidence threshold
def display_result(image_path, confidence_threshold=0.9):
    """
    Display the image and its predicted class with confidence, applying a threshold.
    - If confidence is below the threshold, it's considered invalid.
    """
    try:
        predicted_class, confidence = predict_image(image_path)

        if confidence < confidence_threshold:
            print(f"Prediction Confidence ({confidence * 100:.2f}%) is too low. This is an invalid prediction.")
            return
        
        # Load and display the image
        img = load_img(image_path)
        plt.imshow(img)
        plt.axis('off')
        plt.show()

        # Display prediction
        print(f"Predicted Class: {predicted_class}")
        print(f"Confidence: {confidence * 100:.2f}%")
    
    except Exception as e:
        print(f"Error: {str(e)}")

# Test the function with a sample image (provide the correct path to your test image)
test_image_path = 'media/appar.jpg'  # Change this to your image path
display_result(test_image_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Top 3 Predictions:
1. VI-chickenpox: 82.04%
2. FU-athlete-foot: 15.78%
3. BA- cellulitis: 0.99%
Prediction Confidence (82.04%) is too low. This is an invalid prediction.
